# Исследование надежности заемщиков

## Изучение общей информации о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('...')
except:
    data = pd.read_csv('...')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
#Вывод количества пропущенных значений
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
#Пропуски в столбце "total_income" заполним медианными значениями дохода в зависимости от типа занятости из столбца "income_type"
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
#Замена отрицательных значений дней на положительные
data['days_employed'] = data['days_employed'].abs()

In [7]:
#Медианное значение трудового стажа в днях
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [8]:
#Вывод уникальных значений в столбце "children"
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
#Удалим строки, в которых встречаются аномальные количества детей
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
#Пропуски в столбце "days_employed" заполним медианными значениями стажа в зависимости от типа занятости из столбца "income_type"
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

`Медианное значение является лучшим решением для замены пропусков в количественных переменных, т.к. данный показатель нивелирует возможное искажение данных, получаемое при расчете среднего значения, т.к. аномально большие или маленькие значения выборки могут сильно завысить или занизить среднюю, в результате последующие расчеты на основе такого показателя не будут иметь ничего общего с действительностью. В то время как медиана показывает срединное значение всей выборки, то есть одна половина выборки больше этого значения, а вторая - меньше. Одним из показательных примеров в данном случае может быть расчет средней заработной платы. Если в организации начать увеличивать зарплату только директору, то средняя по организации будет расти, в то время как у остальных сотрудников зарплата не поменяется. А если отслеживать динамику по медиане, то будет видно, что она не меняется.`

In [12]:
#Проверим отсутствие пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

`В исходных данных пропуски были в двух столбцах - количество дней стажа и ежемесячный доход, при чем эти пропуски были в одних и тех же строках. В данном случае пропуски могут быть как полностью случайными, случайными, так и не случайными. Раз пропуски были и в стаже и в доходе, то возможно заемщик не имеет официального трудоустройства и поэтому не мог подтвердить свой стаж и доход и не заполнил эти разделы. А возможно, что это просто была техническая ошибка при выгрузке данных.`

### Изменение типов данных

In [13]:
#Заменим вещественный тип данных на целочисленный
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
#Устраним неявные дубликаты
data['education'] = data['education'].str.lower()

In [15]:
#Проверим наличие строк-дубликатов
data.duplicated().sum()

71

In [16]:
#Удалим строки-дубликаты
data = data.drop_duplicates()

### Категоризация данных

In [17]:
#Присвоим каждому клиенту категорию на основе его дохода
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
#Посмотрим на перечень уникальных целей взятия кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
#Присвоим каждому клиенту категорию на основе его цели взятия кредита
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Зависимость между количеством детей и возвратом кредита в срок

In [22]:
data['debt'].value_counts()

0    19599
1     1732
Name: debt, dtype: int64

In [23]:
data_pivot_children = data.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count')
data_pivot_children

debt,0,1
children,,
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [24]:
data_pivot_children['fraction'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1])
data_pivot_children.sort_values(by='fraction')

debt,0,1,fraction
children,,,
0,13028.0,1063.0,0.075438
3,303.0,27.0,0.081818
1,4364.0,444.0,0.092346
2,1858.0,194.0,0.094542
4,37.0,4.0,0.097561
5,9.0,NaN,NaN


In [25]:
fraction_children = (
    (data_pivot_children[1].sum() - data_pivot_children.loc[0,1]) / 
    ((data_pivot_children[0].sum() - data_pivot_children.loc[0,0]) + 
     (data_pivot_children[1].sum() - data_pivot_children.loc[0,1]))
)
f'Доля просроченных платежей для заемщиков с детьми составляет {fraction_children:.1%}'

'Доля просроченных платежей для заемщиков с детьми составляет 9.2%'

**Вывод:** В целом можно отметить небольшую разницу в доле просроченных платежей среди бездетных заемщиков и заемщиков с детьми. Среди заемщиков без детей доля просроченных платежей составила 7,5%, в то время как среди заемщиков с детьми - 9,2%.
Минимальная доля среди заемщиков с детьми составила 8,2% (для заемщиков с тремя детьми), наиболее массовая категория среди заемщиков с детьми - с одним/двумя детьми - имеет долю просрочек 9,2-9,4%. Наибольшая доля просрочек отмечена среди заемщиков с четырьмя детьми - 9,8%. Среди заемщиков с пятью детьми просрочек не наблюдалось, но их количество в выборке всего 9 единиц, и данный результат не является показательным.

### Зависимость между семейным положением и возвратом кредита в срок

In [26]:
data['family_status'].value_counts()

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [27]:
data_pivot_family = data.pivot_table(index='family_status', columns='debt', values='purpose', aggfunc='count')
data_pivot_family

debt,0,1
family_status,,
Не женат / не замужем,2523,273
в разводе,1105,84
вдовец / вдова,888,63
гражданский брак,3749,385
женат / замужем,11334,927


In [28]:
data_pivot_family['fraction'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1])
data_pivot_family.sort_values(by='fraction')

debt,0,1,fraction
family_status,,,
вдовец / вдова,888,63,0.066246
в разводе,1105,84,0.070648
женат / замужем,11334,927,0.075606
гражданский брак,3749,385,0.093130
Не женат / не замужем,2523,273,0.097639


**Вывод:** Наблюдается, что есть некоторая зависимость между семейным положением и наличием просрочек по кредитам. Заемщики, не состоявшие в официальном браке (гражданский брак и не женат/не замужем), имеют наибольшую долю просрочек - 9,3% и 9,8% соотвественно. Заемщики, состоящие в браке или находящиеся в разводе, имеют меньшую долю просрочек - 7,6% и 7,1% соотвественно. Наименьшую долю имеют вдовы/вдовцы - 6,6%. 

### Зависимость между уровнем дохода и возвратом кредита в срок

In [29]:
data['total_income_category'].value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

In [30]:
data_pivot_income = data.pivot_table(index='total_income_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_income

debt,0,1
total_income_category,,
A,23,2
B,4660,354
C,14568,1353
D,328,21
E,20,2


In [31]:
data_pivot_income['fraction'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1])
data_pivot_income.sort_values(by='fraction')

debt,0,1,fraction
total_income_category,,,
D,328,21,0.060172
B,4660,354,0.070602
A,23,2,0.080000
C,14568,1353,0.084982
E,20,2,0.090909


**Вывод:** Однозначная зависимость между размером дохода и наличием задолженности по кредиту отсутствует. Примерно одинаковый процент просрочек имеют как самые богатые заемщики с доходом более 1 млн рублей (8,0%), так и заемщики с доходом от 50 до 200 тысяч рублей (8,5%). Наибольшая доля просрочек у самой "бедной" категории заемщиков с доходом до 30 тысяч рублей - 9,1%, а у следующей за ними категорией с доходами от 30 до 50 тысяч рублей - уже наименьшая 6,0%.

### Влияние целей кредита на его возврат в срок

In [32]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

In [33]:
data_pivot_purpose = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose

debt,0,1
purpose_category,,
операции с автомобилем,3879,400
операции с недвижимостью,9971,780
получение образования,3619,369
проведение свадьбы,2130,183


In [34]:
data_pivot_purpose['fraction'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1])
data_pivot_purpose.sort_values(by='fraction')

debt,0,1,fraction
purpose_category,,,
операции с недвижимостью,9971,780,0.072551
проведение свадьбы,2130,183,0.079118
получение образования,3619,369,0.092528
операции с автомобилем,3879,400,0.093480


**Вывод:** Можно отметить, что кредиты, выданные с целью получения образования и операций с автомобилем имеют чуть больший процент просроченных платежей (по 9,3%), чем кредиты, выданные на операции с недвижимостью и на проведение свадьбы (7,3% и 7,9% соответственно).

## Общий вывод

С учетом проведенного анализа, наиболее благонадежным является заемщик категории 'вдова/вдовец', 'в разводе', 'женат/замужем', не имеющие детей, с доходом от 30 до 50 тысяч рублей и целью кредита 'на проведение свадьбы' и 'операции с недвижимостью', наиболее неблагонадежным - заемщик категорий 'Не женат / не замужем' и 'гражданский брак', имеющий детей, доход до 30 тысяч рублей и цель кредита 'получение образования' и 'операции с автомобилем'.